In [2]:
using Yao
using Yao.Intrinsics
using Compat.Test

In [3]:
# pack bits to numbers.
packbits(arr::AbstractArray) = slicedim(sum(mapslices(x -> x .* (1 .<< (0:size(arr, 1)-1)), arr, 1), 1), 1, 1)
#packbits(arr::AbstractArray) = reduce(+, mapslices(x -> x .* (1 .<< (0:size(arr, 1)-1)), arr, 1), 1)

packbits (generic function with 1 method)

In [5]:
@test bitarray(2, num_bit=4) == [false, true, false, false]
@test packbits(BitArray([true, true, true])) == 7

@test packbits(bitarray(3, num_bit=10)) == 3
@test packbits(bitarray([5,3,7,21], num_bit=10)) == [5, 3, 7, 21]

Test Passed


In [8]:
# a sample is a bar or a stripe.
is_bar(samples::AbstractMatrix) = sum(abs.(diff(samples, 1)), (1, 2))[] == 0
is_stripe(samples::AbstractMatrix) = sum(abs.(diff(samples, 2)), (1, 2))[] == 0

is_stripe (generic function with 1 method)

In [10]:
ba = reshape(bitarray(7, num_bit=9), 3,3)
@test is_bar(ba) == true
@test is_stripe(ba) == false

ba0 = reshape(bitarray(0, num_bit=9), 3,3)
@test is_bar(ba0) == true
@test is_stripe(ba0) == true

Test Passed


In [12]:
is_bs(arr::AbstractMatrix) = is_bar(arr) || is_stripe(arr)

is_bs (generic function with 1 method)

In [13]:
@test is_bs(ba) == is_bs(ba0) == true

Test Passed


In [14]:
binary_basis(geometry...) = reshape(bitarray(geometry|>prod|>basis|>collect, num_bit=prod(geometry)), (geometry..., :))

binary_basis (generic function with 1 method)

In [15]:
@test size(binary_basis(2,2)) == (2,2,16)

Test Passed


In [22]:
# get bar and stripes PDF
function barstripe_pdf(M::Int, N::Int)
    bss = binary_basis(M, N)
    [bss[:,:,i] |> is_bs for i in indices(bss, 3)] |> normalize
end

barstripe_pdf (generic function with 1 method)

In [32]:
# the total configuration space for bar and stripes.
function bs_configs(M, N)
    bss = binary_basis(M, N)
    slicedim(bss, 3, [bss[:,:,i] |> is_bs for i in indices(bss, 3)])
end

bs_configs (generic function with 2 methods)

In [38]:
@test reshape(bs_configs(2,2), 4, :) |> packbits == findn(barstripe_pdf(2,2))-1

Test Passed


In [50]:
bdistance(i::DInt, j::DInt) = count_ones(i⊻j)

bdistance (generic function with 1 method)

In [51]:
@test bdistance(1,7) == 2

Test Passed
